# Elementary Recipe for Quantum Circuit Unoptimization

## Preliminaries

In [1]:
import numpy as np
import random

from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, random_unitary

First, we need a way of generating a random circuit consisting of two-qubit gates.

In [2]:
def random_two_qubit_circuit(num_qubits: int, depth: int) -> QuantumCircuit:
    """Generate a random quantum circuit with two-qubit gates.

    Args:
        num_qubits: The number of qubits in the circuit.
        depth: The number of layers (depth) of two-qubit gates.

    Returns:
        The generated random circuit.
    """
    qc = QuantumCircuit(num_qubits)

    # Map gate names to their corresponding methods
    gate_map = {
        "cx": qc.cx,
        "cz": qc.cz,
        "swap": qc.swap,
        "iswap": qc.iswap
    }

    for _ in range(depth):
        qubit1, qubit2 = random.sample(range(num_qubits), 2)
        gate = random.choice(list(gate_map.keys()))
        gate_map[gate](qubit1, qubit2)

    return qc

As an example, here is randomly generated 5-qubit and depth-10 circuit consisting of two-qubit gates 

In [3]:
num_qubits = 5
depth = 10
random_circuit = random_two_qubit_circuit(num_qubits, depth)

print(random_circuit)

                                    ┌───┐   ┌────────┐
q_0: ────────────■──────────────────┤ X ├───┤1       ├
     ┌────────┐  │                  └─┬─┘   │        │
q_1: ┤0       ├──■─────────────■──■───┼───X─┤  Iswap ├
     │        │     ┌────────┐ │  │   │   │ │        │
q_2: ┤  Iswap ├──■──┤1       ├─■──┼───┼───┼─┤0       ├
     │        │┌─┴─┐│  Iswap │    │   │   │ └────────┘
q_3: ┤1       ├┤ X ├┤0       ├─■──┼───■───┼───────────
     └────────┘└───┘└────────┘ │  │       │           
q_4: ──────────────────────────■──■───────X───────────
                                                      


## Elementary recipe

The elementary recipe (ER) for quantum circuit unoptimization is given by Figure-1 in [arXiv:2311.03805](https://arxiv.org/pdf/2311.03805). The ER contains the following steps (applied in the following order):

1. Gate insertion
2. Gate swapping
3. Gate decomposition
4. Gate synthesis

### Gate insertion

In [4]:
def gate_insert(qc: QuantumCircuit) -> QuantumCircuit:
    """Insert a two-qubit gate A and its Hermitian conjugate A† between two gates B1 and B2 that share a common qubit.

    Args:
        qc: The input quantum circuit.

    Returns:
        The modified quantum circuit with A and A† inserted.
    """
    # Collect all two-qubit gates with their indices and qubits
    two_qubit_gates = []
    for idx, instruction in enumerate(qc.data):
        instr = instruction.operation
        qargs = instruction.qubits
        cargs = instruction.clbits
        if len(qargs) == 2:
            qubit_indices = [qc.find_bit(qarg).index for qarg in qargs]
            two_qubit_gates.append({'index': idx, 'qubits': qubit_indices})

    # Find a pair of gates that share a common qubit
    found_pair = False
    for i in range(len(two_qubit_gates)):
        for j in range(i+1, len(two_qubit_gates)):
            qubits_i = set(two_qubit_gates[i]['qubits'])
            qubits_j = set(two_qubit_gates[j]['qubits'])
            common_qubits = qubits_i & qubits_j
            if len(common_qubits) == 1:
                B1_idx = two_qubit_gates[i]['index']
                B2_idx = two_qubit_gates[j]['index']
                B1_qubits = two_qubit_gates[i]['qubits']
                B2_qubits = two_qubit_gates[j]['qubits']
                shared_qubit = list(common_qubits)[0]
                found_pair = True
                break
        if found_pair:
            break

    if not found_pair:
        raise ValueError("No pair of two-qubit gates sharing a common qubit found.")

    # Create a new circuit
    new_qc = QuantumCircuit(qc.num_qubits)

    # Copy the gates up to and including B1
    for instruction in qc.data[:B1_idx+1]:
        instr = instruction.operation
        qargs = instruction.qubits
        cargs = instruction.clbits
        new_qc.append(instr, qargs, cargs)

    # Generate a random two-qubit unitary A
    A = random_unitary(4)
    A_dag = A.adjoint()

    # Decide on which qubits to apply A and A†
    # Choose the shared qubit and one of the other qubits
    other_qubits = list((set(B1_qubits + B2_qubits) - {shared_qubit}))
    target_qubit = other_qubits[0]  # Choose one of the other qubits

    # Map indices back to qubits
    qubit_map = {qc.find_bit(q).index: q for q in qc.qubits}
    qubits_for_A = [qubit_map[shared_qubit], qubit_map[target_qubit]]

    # Insert A and A†
    new_qc.unitary(A, qubits_for_A, label='A')
    new_qc.unitary(A_dag, qubits_for_A, label='A†')

    # Copy the remaining gates
    for instruction in qc.data[B1_idx+1:]:
        instr = instruction.operation
        qargs = instruction.qubits
        cargs = instruction.clbits
        new_qc.append(instr, qargs, cargs)

    return new_qc

In [5]:
# Example usage
qc = random_two_qubit_circuit(4, 5)
new_qc = gate_insert(qc)
print(new_qc)

                                       
q_0: ────────────■────────────────X────
     ┌────────┐  │  ┌────┐┌─────┐ │    
q_1: ┤0       ├──┼──┤1   ├┤1    ├─X────
     │        │┌─┴─┐│    ││     │      
q_2: ┤  Iswap ├┤ X ├┤  A ├┤  A† ├─■──■─
     │        │└───┘│    ││     │ │  │ 
q_3: ┤1       ├─────┤0   ├┤0    ├─■──■─
     └────────┘     └────┘└─────┘      


### Gate swapping

In [ ]:
def gate_swap():
    pass

### Gate decomposition

In [ ]:
def gate_decompose():
    pass

### Gate synthesis

In [ ]:
def gate_synthesize():
    pass